In [17]:
import pandas as pd
import numpy as np
import pylab as pl
from sklearn import ensemble
from sklearn import datasets
from sklearn.utils import shuffle
from sklearn.metrics import mean_squared_error

from util.methods import Methods
from util.receiving import Receiving
from util.stat import FantasyStats

from cost_functions import CostFunction
pd.set_option('display.width', 300)
# Update this to the latest year that data is available
score_year = 2014

In [18]:
func = Methods()
receiving = Receiving(2002, score_year)
receiving.compile_data()
receiving.construct_feature_vector()
df=receiving.get_receiving_data()
df[receiving.dep_var + receiving.feature_vector].describe()

,y_future_rank,y_top5,y_sqrt_future_rank,y_future_points,y_sqr_future_points,RK,is_rec,games_played,n_years,td_per_game,...,percent_comp,avg_career_rank,previous_rank1_clean,previous_rank2_clean,previous_rank3_clean,previous_rank4_clean,previous_rank5_clean,lst_rank_delta,r_td_rec,log_yards
count,377.000000,602.000000,377.000000,377.000000,377.000000,602.000000,602.000000,602.000000,602.000000,602.000000,...,602.000000,602.000000,602.000000,602.000000,602.000000,602.000000,602.000000,602.000000,602.000000,602.000000
mean,27.328912,0.131229,4.899602,177.185676,13.194397,30.564784,0.807309,14.812292,3.686047,0.412498,...,0.648074,30.336379,29.845515,30.049834,30.275748,30.460133,30.822259,0.668863,0.087938,6.818141
std,17.141248,0.337932,1.825282,47.894304,1.761189,17.377571,0.394740,1.335419,2.577150,0.220188,...,0.136200,14.485071,17.289097,17.276575,17.073796,16.902458,16.801077,2.912282,0.043636,0.240815
min,1.000000,0.000000,1.000000,92.000000,9.591663,1.000000,0.000000,7.000000,1.000000,0.000000,...,0.393333,3.000000,1.000000,1.000000,1.000000,1.000000,1.000000,-0.977273,0.000000,6.342121
25%,12.000000,0.000000,3.464102,139.000000,11.789826,16.000000,1.000000,15.000000,1.000000,0.250000,...,0.562500,19.500000,15.000000,15.000000,16.000000,16.000000,17.000000,-0.221545,0.055751,6.629363
50%,26.000000,0.000000,5.099020,168.000000,12.961481,31.000000,1.000000,15.000000,3.000000,0.375000,...,0.617631,29.000000,30.000000,30.000000,30.000000,30.000000,31.000000,0.000000,0.083333,6.774224
75%,41.000000,0.000000,6.403124,209.000000,14.456832,46.000000,1.000000,16.000000,5.000000,0.533333,...,0.679889,42.000000,44.000000,44.000000,44.000000,44.000000,44.000000,0.416159,0.112846,6.987490
max,60.000000,1.000000,7.745967,347.000000,18.627936,60.000000,1.000000,16.000000,13.000000,1.437500,...,1.000000,60.000000,60.000000,60.000000,60.000000,60.000000,60.000000,39.000000,0.250000,7.582738


In [19]:
stat = FantasyStats(df, score_year)
stat.build_train_valid_score(60)
df_train = stat.get_df_train()
print len(df_train)

df_valid = stat.get_df_valid()
print len(df_valid)

df_score = stat.get_df_score()
print len(df_score)

327
215
60


### Train the Gradient Boosted Regression

In [30]:
params = {'n_estimators': 1000,
          'max_depth': 3,
          'min_samples_split': 2,
          'learning_rate': 0.04,
          'loss': 'ls',
          'verbose': 1}

dependent_variable = receiving.dep_var[2]
print dependent_variable

stat.train_gbr_fulL_sample(dependent_variable, params, receiving.feature_vector)

clfr = stat.get_gbr_result()
print "\n"
stat.feature_importance(clfr, receiving.feature_vector)

y_sqrt_future_rank
      Iter       Train Loss   Remaining Time 
         1           3.2533            1.57s
         2           3.1888            1.28s
         3           3.1287            1.25s
         4           3.0743            1.25s
         5           3.0170            1.22s
         6           2.9627            1.16s
         7           2.9153            1.14s
         8           2.8694            1.11s
         9           2.8269            1.07s
        10           2.7880            1.03s
        20           2.4538            0.98s
        30           2.2042            0.89s
        40           2.0318            0.83s
        50           1.8954            0.80s
        60           1.8148            0.76s
        70           1.7334            0.74s
        80           1.6480            0.72s
        90           1.5664            0.71s
       100           1.4832            0.69s
       200           0.9732            0.56s
       300           0.6314        

### Train the Sqrt Points Gradient Boosted Regression

In [22]:
params = {'n_estimators': 500,
          'max_depth': 3,
          'min_samples_split': 2,
          'learning_rate': 0.04,
          'loss': 'ls',
          'verbose': 1}

dependent_variable = receiving.dep_var[4]
print dependent_variable

stat.train_gbr_fulL_sample(dependent_variable, params, receiving.feature_vector)

clfr2 = stat.get_gbr_result()
print "\n"
stat.feature_importance(clfr2, receiving.feature_vector)

y_sqr_future_points
      Iter       Train Loss   Remaining Time 
         1           3.0236            0.75s
         2           2.9585            0.63s
         3           2.8981            0.59s
         4           2.8418            0.58s
         5           2.7899            0.59s
         6           2.7410            0.58s
         7           2.6953            0.56s
         8           2.6488            0.55s
         9           2.6083            0.53s
        10           2.5702            0.52s
        20           2.2359            0.45s
        30           2.0056            0.42s
        40           1.8303            0.39s
        50           1.6896            0.37s
        60           1.5940            0.35s
        70           1.5171            0.34s
        80           1.4401            0.33s
        90           1.3802            0.32s
       100           1.3185            0.31s
       200           0.8608            0.21s
       300           0.5834       

### Train the Gradient Boosted Classifier

In [23]:
params = {'n_estimators': 400, 
          'learning_rate': 0.04,
          'random_state': 0,
          'max_depth': 3,
          'min_samples_split': 2,
          'verbose': 1, 
          'random_state': 0}
 

dependent_variable = receiving.dep_var[1]
print dependent_variable
stat.train_gbc_full_sample(dependent_variable, params, receiving.feature_vector)
print "\n"
clfc = stat.get_gbc_result()
stat.feature_importance(clfc, receiving.feature_vector)

y_top5
      Iter       Train Loss   Remaining Time 
         1           0.7555            0.83s
         2           0.7379            0.68s
         3           0.7232            0.66s
         4           0.7091            0.64s
         5           0.6968            0.63s
         6           0.6861            0.62s
         7           0.6756            0.60s
         8           0.6668            0.58s
         9           0.6579            0.57s
        10           0.6491            0.59s
        20           0.5848            0.54s
        30           0.5383            0.51s
        40           0.5051            0.52s
        50           0.4789            0.50s
        60           0.4543            0.48s
        70           0.4338            0.46s
        80           0.4173            0.44s
        90           0.3990            0.42s
       100           0.3830            0.40s
       200           0.2535            0.26s
       300           0.1737            0.13s
  

### Validation sample (boosted reg 1)

In [31]:
df_combined = stat.score_train_and_validation_reg(clfr, receiving.feature_vector, True)
df_combined[df_combined.year==score_year-1].sort('p_rank', ascending=True)[['PLAYER', 'TEAM', 'y_future_rank', 'p_rank']][:15]

,PLAYER,TEAM,y_future_rank,p_rank
446,"Antonio Brown, WR",PIT,1,1
533,"Demaryius Thomas, WR",DEN,2,2
327,"Jordy Nelson, WR",GB,4,3
148,"Josh Gordon, WR",CLE,NaN,4
412,"Dez Bryant, WR",DAL,8,5
538,"Golden Tate, WR",SEA,7,6
280,"Emmanuel Sanders, WR",PIT,5,7
564,"T.Y. Hilton, WR",IND,6,8
171,"Cecil Shorts III, WR",JAX,NaN,9
461,"DeAndre Hopkins, WR",HOU,12,10


### Validation sample (boosted reg 2)

In [32]:
df_combined = stat.score_train_and_validation_reg(clfr2, receiving.feature_vector, False)
df_combined[df_combined.year==score_year-1].sort('p_rank', ascending=True)[['PLAYER', 'TEAM', 'y_future_rank', 'p_rank']][:15]

,PLAYER,TEAM,y_future_rank,p_rank
446,"Antonio Brown, WR",PIT,1,1
533,"Demaryius Thomas, WR",DEN,2,2
412,"Dez Bryant, WR",DAL,8,3
327,"Jordy Nelson, WR",GB,4,4
148,"Josh Gordon, WR",CLE,NaN,5
458,"Alshon Jeffery, WR",CHI,14,6
280,"Emmanuel Sanders, WR",PIT,5,7
299,"Calvin Johnson, WR",DET,16,8
341,"Wes Welker, WR",DEN,NaN,9
564,"T.Y. Hilton, WR",IND,6,10


### Validation sample (boosted classifier 1)

In [33]:
df_combined = stat.score_train_and_validation_class(clfc, receiving.feature_vector, False)
df_combined[df_combined.year==score_year-1].sort('p_rank', ascending=True)[['PLAYER', 'TEAM', 'y_future_rank', 'p_rank']][:15]

,PLAYER,TEAM,y_future_rank,p_rank
446,"Antonio Brown, WR",PIT,1,1
412,"Dez Bryant, WR",DAL,8,2
533,"Demaryius Thomas, WR",DEN,2,3
327,"Jordy Nelson, WR",GB,4,4
564,"T.Y. Hilton, WR",IND,6,5
538,"Golden Tate, WR",SEA,7,6
280,"Emmanuel Sanders, WR",PIT,5,7
206,"A.J. Green, WR",CIN,20,8
458,"Alshon Jeffery, WR",CHI,14,9
236,"Brandon Marshall, WR",CHI,59,10


In [34]:
def _final_order_score(df):
    score = float(df['pred_boost_reg1']) * 0.4 +\
            float(df['pred_boost_reg2']) * 0.4 +\
            float(df['pred_classifier1']) * 0.2
    return score

In [36]:
df_score_reg = pd.DataFrame(stat.score_pred_reg(clfr, receiving.feature_vector, True))
df_score_reg.set_index('PLAYER', inplace=True, drop=False)
df_score_reg.rename(columns={'p_rank': 'pred_boost_reg1'}, inplace=True)

df_score_reg2 = pd.DataFrame(stat.score_pred_reg(clfr2, receiving.feature_vector, False))
df_score_reg2.set_index('PLAYER', inplace=True, drop=False)
df_score_reg2.rename(columns={'p_rank': 'pred_boost_reg2'}, inplace=True)


df_score_class = pd.DataFrame(stat.score_pred_class(clfc, receiving.feature_vector, False))
df_score_class.set_index('PLAYER', inplace=True, drop=False)
df_score_class.rename(columns={'p_rank': 'pred_classifier1'}, inplace=True)

df_score_all = df_score_reg.join(df_score_class['pred_classifier1'])
df_score_all = df_score_all.join(df_score_reg2['pred_boost_reg2'])

df_score_all['final_order_score'] = df_score_all.apply(_final_order_score, axis=1)
df_score_all.sort('final_order_score', ascending=True, inplace=True)
df_score_all.reset_index(inplace=True, drop=True)
df_score_all.rename(columns={'RK': '%s Rank' % score_year}, inplace=True)

df_score_all[['PLAYER', 'TEAM', '%s Rank' % score_year, 'final_order_score', 'pred_boost_reg1', 'pred_boost_reg2', 'pred_classifier1']]

,PLAYER,TEAM,2014 Rank,final_order_score,pred_boost_reg1,pred_boost_reg2,pred_classifier1
0,"T.Y. Hilton, WR",IND,6,2.6,2,3,3
1,"Julio Jones, WR",ATL,3,3.0,1,4,5
2,"Jordy Nelson, WR",GB,4,4.8,3,1,16
3,"Antonio Brown, WR",PIT,1,7.8,4,9,13
4,"Alshon Jeffery, WR",CHI,14,8.6,9,12,1
5,"Demaryius Thomas, WR",DEN,2,8.6,6,6,19
6,"Mike Wallace, WR",MIA,38,8.6,5,8,17
7,"Odell Beckham Jr., WR",NYG,10,8.6,11,7,7
8,"Randall Cobb, WR",GB,11,9.2,8,5,20
9,"Rob Gronkowski, TE",NE,15,11.4,7,2,39
